In [1]:
import os
project_path = '/workspace/'
os.chdir(project_path)

In [2]:
!pip install hydra-core --upgrade
!pip install hdf5plugin

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
!sudo apt-get update && sudo apt-get install -y libgl1 libglib2.0-0


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1411 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2306 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2738 kB]
Fetched 6712 kB in 4s (1889 kB/s)                        
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1 is already the newest version (1.4.0-1).
libglib2.0-0 is already the newest version (2.72.4-0ubuntu2.3).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgrade

In [6]:
!whoami

ubuntu


In [8]:
ls -la

total 104
drwxr-xr-x   1 root   root  4096 Jul 16 08:31 ./
drwxr-xr-x   1 root   root  4096 Jul 16 08:31 ../
-rwxr-xr-x   1 root   root     0 Jul 16 08:31 .dockerenv*
-rw-r--r--   1 root   root 17294 Apr 22 23:41 NGC-DL-CONTAINER-LICENSE
lrwxrwxrwx   1 root   root     7 Apr 10 14:03 bin -> usr/bin/
drwxr-xr-x   2 root   root  4096 Apr 18  2022 boot/
-rw-r--r--   1 root   root  4332 Apr 22 23:42 cuda-keyring_1.1-1_all.deb
drwxr-xr-x   5 root   root   360 Jul 16 08:31 dev/
drwxr-xr-x   1 root   root  4096 Jul 16 15:19 etc/
drwxr-xr-x   1 root   root  4096 Jul 16 08:29 home/
lrwxrwxrwx   1 root   root     7 Apr 10 14:03 lib -> usr/lib/
lrwxrwxrwx   1 root   root     9 Apr 10 14:03 lib32 -> usr/lib32/
lrwxrwxrwx   1 root   root     9 Apr 10 14:03 lib64 -> usr/lib64/
lrwxrwxrwx   1 root   root    10 Apr 10 14:03 libx32 -> usr/libx32/
drwxr-xr-x   2 root   root  4096 Apr 10 14:03 media/
drwxr-xr-x   2 root   root  4096 Apr 10 14:03 mnt/
drwxr-xr-x   1 root   root  4096 Apr 22 23:46 opt/
dr-x

In [9]:
cd /workspace

/workspace


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [11]:
%env HYDRA_FULL_ERROR=1
!python /workspace/main.py

env: HYDRA_FULL_ERROR=1
{'dataset_path': 'data', 'seed': 42, 'num_epoch': 10, 'data_loader': {'common': {'num_voxel_bins': 15}, 'train': {'batch_size': 8, 'shuffle': False}, 'test': {'batch_size': 1, 'shuffle': False}}, 'train': {'no_batch_norm': False, 'initial_learning_rate': 0.01, 'weight_decay': 0.0001, 'epochs': 10}}
on epoch: 1
  0%|                                                   | 0/252 [00:00<?, ?it/s]/home/ubuntu/.local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
batch 0 loss: 195.50781512976658
100%|█████████████████████████████████████████| 252/252 [27:11<00:00,  6.47s/it]
Epoch 1, Loss: 15.087702356454628
Model saved to checkpoints/model_20240716155708.pth
on epoch: 2
100%|█

In [5]:
import torch
import hydra
from omegaconf import DictConfig
from torch.utils.data import DataLoader
import random
import numpy as np
from src.models.evflownet import EVFlowNet
from src.datasets import DatasetProvider
from enum import Enum, auto
from src.datasets import train_collate
from tqdm import tqdm
from pathlib import Path
from typing import Dict, Any
import os
import time

In [6]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

with initialize_config_dir(version_base=None, config_dir="/workspace/configs"):
    args = compose(config_name="base")

print(OmegaConf.to_yaml(args))

dataset_path: data
seed: 42
num_epoch: 1
data_loader:
  common:
    num_voxel_bins: 15
  train:
    batch_size: 8
    shuffle: false
  test:
    batch_size: 1
    shuffle: false
train:
  no_batch_norm: false
  initial_learning_rate: 0.01
  weight_decay: 0.0001
  epochs: 10



In [7]:
class RepresentationType(Enum):
    VOXEL = auto()
    STEPAN = auto()

def set_seed(seed):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)

def compute_epe_error(pred_flow: torch.Tensor, gt_flow: torch.Tensor):
    '''
    end-point-error (ground truthと予測値の二乗誤差)を計算
    pred_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 予測したオプティカルフローデータ
    gt_flow: torch.Tensor, Shape: torch.Size([B, 2, 480, 640]) => 正解のオプティカルフローデータ
    '''
    epe = torch.mean(torch.mean(torch.norm(pred_flow - gt_flow, p=2, dim=1), dim=(1, 2)), dim=0)
    return epe

def save_optical_flow_to_npy(flow: torch.Tensor, file_name: str):
    '''
    optical flowをnpyファイルに保存
    flow: torch.Tensor, Shape: torch.Size([2, 480, 640]) => オプティカルフローデータ
    file_name: str => ファイル名
    '''
    np.save(f"{file_name}.npy", flow.cpu().numpy())


In [ ]:


# Create the directory if it doesn't exist
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

set_seed(args.seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
'''
    ディレクトリ構造:

    data
    ├─test
    |  ├─test_city
    |  |    ├─events_left
    |  |    |   ├─events.h5
    |  |    |   └─rectify_map.h5
    |  |    └─forward_timestamps.txt
    └─train
        ├─zurich_city_11_a
        |    ├─events_left
        |    |       ├─ events.h5
        |    |       └─ rectify_map.h5
        |    ├─ flow_forward
        |    |       ├─ 000134.png
        |    |       |.....
        |    └─ forward_timestamps.txt
        ├─zurich_city_11_b
        └─zurich_city_11_c
    '''

# ------------------
#    Dataloader
# ------------------
loader = DatasetProvider(
    dataset_path=Path(args.dataset_path),
    representation_type=RepresentationType.VOXEL,
    delta_t_ms=100,
    num_bins=4
)
train_set = loader.get_train_dataset()
test_set = loader.get_test_dataset()
collate_fn = train_collate
train_data = DataLoader(train_set,
                                batch_size=args.data_loader.train.batch_size,
                                shuffle=args.data_loader.train.shuffle,
                                collate_fn=collate_fn,
                                drop_last=False)
test_data = DataLoader(test_set,
                                batch_size=args.data_loader.test.batch_size,
                                shuffle=args.data_loader.test.shuffle,
                                collate_fn=collate_fn,
                                drop_last=False)

'''
train data:
    Type of batch: Dict
    Key: seq_name, Type: list
    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
    Key: flow_gt, Type: torch.Tensor, Shape: torch.Size([Batch, 2, 480, 640]) => オプティカルフローデータのバッチ
    Key: flow_gt_valid_mask, Type: torch.Tensor, Shape: torch.Size([Batch, 1, 480, 640]) => オプティカルフローデータのvalid. ベースラインでは使わない

test data:
    Type of batch: Dict
    Key: seq_name, Type: list
    Key: event_volume, Type: torch.Tensor, Shape: torch.Size([Batch, 4, 480, 640]) => イベントデータのバッチ
'''

In [10]:

# ------------------
#       Model
# ------------------
model = EVFlowNet(args.train).to(device)
model.load_state_dict(torch.load("/workspace/checkpoints/model_20240716143432.pth", map_location=device))


# ------------------
#   optimizer
# ------------------
optimizer = torch.optim.Adam(model.parameters(), lr=args.train.initial_learning_rate, weight_decay=args.train.weight_decay)

# ------------------
#   Start training
# ------------------
model.train()
for epoch in range(args.train.epochs):
    total_loss = 0
    print("on epoch: {}".format(epoch+1))
    for i, batch in enumerate(tqdm(train_data)):
        batch: Dict[str, Any]
        event_image = batch["event_volume"].to(device) # [B, 4, 480, 640]
        ground_truth_flow = batch["flow_gt"].to(device) # [B, 2, 480, 640]
        flow = model(event_image) # [B, 2, 480, 640]
        loss: torch.Tensor = compute_epe_error(flow, ground_truth_flow)
        print(f"batch {i} loss: {loss.item()}")
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_data)}')
    
    current_time = time.strftime("%Y%m%d%H%M%S")
    model_path = f"checkpoints/model_{current_time}.pth"
    torch.save(model.state_dict(), model_path)
    print(f"Model saved to {model_path}")


# ------------------
#   Start predicting
# ------------------
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
flow: torch.Tensor = torch.tensor([]).to(device)
with torch.no_grad():
    print("start test")
    for batch in tqdm(test_data):
        batch: Dict[str, Any]
        event_image = batch["event_volume"].to(device)
        batch_flow = model(event_image) # [1, 2, 480, 640]
        flow = torch.cat((flow, batch_flow), dim=0)  # [N, 2, 480, 640]
    print("test done")
# ------------------
#  save submission
# ------------------
file_name = "submission"
save_optical_flow_to_npy(flow, file_name)

on epoch: 1


  0%|          | 0/252 [00:00<?, ?it/s]

batch 0 loss: 7.482555706699425


  0%|          | 1/252 [00:08<36:47,  8.80s/it]

batch 1 loss: 59.47175526413956


...

batch 37 loss: 5.857844467533957


 15%|█▌        | 38/252 [04:41<27:02,  7.58s/it]

batch 38 loss: 9.106228291084589


 15%|█▌        | 39/252 [04:55<26:52,  7.57s/it]


KeyboardInterrupt: 

In [13]:
current_time = time.strftime("%Y%m%d%H%M%S")
model_path = f"checkpoints/model_{current_time}.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")


Model saved to checkpoints/model_20240716151915.pth


In [12]:

# Create the directory if it doesn't exist
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

current_time = time.strftime("%Y%m%d%H%M%S")
model_path = f"checkpoints/model_{current_time}.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

# ------------------
#   Start predicting
# ------------------
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
flow: torch.Tensor = torch.tensor([]).to(device)
with torch.no_grad():
    print("start test")
    for batch in tqdm(test_data):
        batch: Dict[str, Any]
        event_image = batch["event_volume"].to(device)
        batch_flow = model(event_image) # [1, 2, 480, 640]
        flow = torch.cat((flow, batch_flow), dim=0)  # [N, 2, 480, 640]
    print("test done")
# ------------------
#  save submission
# ------------------
file_name = "submission"
save_optical_flow_to_npy(flow, file_name)

Model saved to checkpoints/model_20240716151827.pth
start test


100%|██████████| 97/97 [00:29<00:00,  3.28it/s]


test done
